# DAY 2 : RAG

This is an example for all steps in a basic RAG solution 
steps: 

1. Setup :  chunk/embed/store
2. R-etrival :  embed & do similarity search
3. A -ugmnet :  optionally rerank & add to response
4. G - enerate : ask LLm to answer question based on the retrieved chunks

Questions use #generative-ai-users  or #igiu-innovation-lab slack channel


### Set up variables

In [79]:
from oci.generative_ai_inference import GenerativeAiInferenceClient
from oci.generative_ai_inference.models import OnDemandServingMode, EmbedTextDetails,CohereChatRequest, ChatDetails
import oracledb
import array
import oci
import sys
import os 



CONFIG_PROFILE = "DEFAULT"
compartmentId = "ocid1.compartment.oc1..aaaaaaaa5wmdeu3rf5s4rs4l66rksphne2orz4buauniiqtar63du6ni7icq"
endpointId= "https://inference.generativeai.us-chicago-1.oci.oraclecloud.com"

print (" change the table names to your name & uncomment ")
#sys.exit(1)

DB_CONFIG_DIR = os.path.expanduser("~/.workshop/inno_wallet")
DB_USER = "ONNOVATE"
DB_PASS = "Welcome_2025"
DB_DSN = "innovationlab_medium"
DB_WALLET_LOC=os.path.expanduser("~/.workshop/inno_wallet")
DB_WALLET_PASS="41_innovation"

 change the table names to your name & uncomment 


## chunks we we want to query against 

In [80]:
sentences = ["Baseball is a great game ",
				"baseball games typically last 9 innings",
				"Baseball game can finish in about 2 hours",
				"Indias favroite passtime is cricket",
				"England's favorite passtime is football",
				"Football is called soccer in America",
				"baseball is americas favroite pass time sport"]

## open database connection

In [81]:
db = oracledb.connect( config_dir= DB_CONFIG_DIR, user= DB_USER, password=DB_PASS, dsn=DB_DSN,wallet_location=DB_WALLET_LOC, wallet_password=DB_WALLET_PASS)
cursor = db.cursor()


## create tables 

In [86]:
sql = [
		"""drop table if exists asagarwa2_embedding purge"""	,
  
		"""
 		create table asagarwa2_embedding (
   			id number,
			text varchar2(4000),
			vec vector,
			primary key (id)
		)"""
	]
 
for s in sql : 
	cursor.execute(s)

db.commit()

### load the profile

In [87]:
# oci key enabled for api access
config = oci.config.from_file('~/.oci/workshop', CONFIG_PROFILE)

### set up LLM client 

In [88]:

# create a llm client 
llm_client = GenerativeAiInferenceClient(
				config=config, 
				service_endpoint=endpointId, 
				retry_strategy=oci.retry.NoneRetryStrategy(),
				timeout=(10,240))	

## Create embeddings

In [89]:
embed_text_detail = EmbedTextDetails()
embed_text_detail.serving_mode = OnDemandServingMode(model_id="cohere.embed-english-v3.0")
embed_text_detail.truncate = embed_text_detail.TRUNCATE_END
embed_text_detail.input_type = embed_text_detail.INPUT_TYPE_SEARCH_DOCUMENT
embed_text_detail.compartment_id = compartmentId
embed_text_detail.inputs = sentences 

response = llm_client.embed_text(embed_text_detail)
embeddings = response.data.embeddings


## insert embedding in database

In [90]:
for i in range(len(embeddings)):
    cursor.execute("insert into asagarwa2_embedding values (:1, :2, :3)", 
                   [i, sentences[i], array.array("f",embeddings[i])])
    print(f"inserted {i}-{sentences[i]}")

print("commiting")
db.commit()

inserted 0-Baseball is a great game 
inserted 1-baseball games typically last 9 innings
inserted 2-Baseball game can finish in about 2 hours
inserted 3-Indias favroite passtime is cricket
inserted 4-England's favorite passtime is football
inserted 5-Football is called soccer in America
inserted 6-baseball is americas favroite pass time sport
commiting


## read the table 

In [91]:
cursor.execute('select id,text from asagarwa2_embedding')
for row in cursor:
	print(f"{row[0]}:{row[1]}")

2:Baseball game can finish in about 2 hours
3:Indias favroite passtime is cricket
4:England's favorite passtime is football
0:Baseball is a great game 
1:baseball games typically last 9 innings
5:Football is called soccer in America
6:baseball is americas favroite pass time sport


## Ask A question to answer

In [92]:
query = input("Ask a question: ").strip().lower()
q=[]
q.append(query)

## embed the query

we nede to do the "R" part of rag - retrieve.  we retrieve in following steps
1. embed the query test
1. do a similarity serach to find the text similar to it 
2. optionally rerank it 

In [93]:

# embed

embed_text_detail.inputs = q

response = llm_client.embed_text(embed_text_detail)
vec = array.array("f",response.data.embeddings[0])




In [94]:
# simialrity search of embedded text 
 
# There are multiple search algorithms: COSINE, DOT, EUCLIDEAN
cursor.execute("""
		select id,text, vector_distance(vec, :1, EUCLIDEAN) d 
		from asagarwa2_embedding
		order by d
		fetch first 10 rows only
	""", [vec])

rows =[]
for row in cursor:
	r = [row[0], row[1], row[2]]
	print(r)
	rows.append(r)

print (rows)

[3, 'Indias favroite passtime is cricket', 0.8447505235671997]
[6, 'baseball is americas favroite pass time sport', 0.9569643139839172]
[4, "England's favorite passtime is football", 0.9846054911613464]
[0, 'Baseball is a great game ', 1.0166956186294556]
[5, 'Football is called soccer in America', 1.0340462923049927]
[1, 'baseball games typically last 9 innings', 1.0601223707199097]
[2, 'Baseball game can finish in about 2 hours', 1.1067509651184082]
[[3, 'Indias favroite passtime is cricket', 0.8447505235671997], [6, 'baseball is americas favroite pass time sport', 0.9569643139839172], [4, "England's favorite passtime is football", 0.9846054911613464], [0, 'Baseball is a great game ', 1.0166956186294556], [5, 'Football is called soccer in America', 1.0340462923049927], [1, 'baseball games typically last 9 innings', 1.0601223707199097], [2, 'Baseball game can finish in about 2 hours', 1.1067509651184082]]


### optionally rerank

In [ ]:
# look at cohere reranking example 

## A of RAG : augment

we attach the retrieved chucks as documents to chat request 

In [95]:
# prepare the payloafd 
# chat request 
cohere_chat_request = CohereChatRequest()
#cohere_chat_request.preamble_override = "you always answer in a one stanza poem."
cohere_chat_request.is_stream = False 
cohere_chat_request.max_tokens = 500
cohere_chat_request.temperature = 0.75
cohere_chat_request.top_p = 0.7
cohere_chat_request.frequency_penalty = 1.0
#cohere_chat_request.documents = get_documents()

#chat detail 
chat_detail = ChatDetails()
chat_detail.serving_mode = OnDemandServingMode(model_id="cohere.command-r-plus")
chat_detail.compartment_id = compartmentId
chat_detail.chat_request = cohere_chat_request



In [96]:
# add documents
docs =[]
for row in rows:
    doc = {
        "title" : f"document {row[0]}",
        "snippet" : row[1]
    } 
    docs.append(doc)

cohere_chat_request.documents = docs



## G in RAG : Generate the respone

In [97]:
cohere_chat_request.message = query
cohere_chat_request.preamble_override = " answer only from selected docs, ignore any other information you may know"
llm_response = llm_client.chat(chat_detail)
print ("query executed")

query executed


## print the response 

In [98]:
print("**************************Chat Result**************************")
print(query)
llm_text = llm_response.data.chat_response.text
print(llm_response.data.chat_response.text)
print("************************** Citations**************************")
print(llm_response.data.chat_response.citations)

**************************Chat Result**************************
which country is cricket popular?
Cricket is a popular sport in India.
************************** Citations**************************
[{
  "document_ids": [
    "doc_0"
  ],
  "end": 36,
  "start": 30,
  "text": "India."
}]


## close the dabasae connections

In [ ]:
cursor.close()
db.close()


## Exercise
 * Implement a “talk-to-document”
 * Try one of :
    * Text document
    * PDF document
 * Play with following features
    * Chunking
    * Different types similarity search
    * Reranking
    * Citations
